In [29]:
import gradio as gr
from Question import Question_generae
import erniebot
import requests
import wget
from PIL import Image
import json
import os
import uuid
from pathlib import Path
if not os.path.exists('./voice'):
    os.mkdir('./voice')
    os.mkdir('./img')

In [30]:
enrine_key = "e9dc9bdd67bf5284156b8173d1b8d007831b4198"
# 语音生成的key, 获取地址：https://console.bce.baidu.com/ai/?fromai=1#/ai/speech/overview/index
speak_client_id = '6aD2xRhkWnaDlc9mAnsPuQOZ'
speak_client_secret = 'oDRnDURL3GrRO0EXPajTGWyF70hdAy0V'
# 图片生成的key, 获取地址：https://console.bce.baidu.com/ai/#/ai/intelligentwriting/app/list
draw_api_key = "a3DfIwgXjAuYeEinhTRDXycl"
# draw_api_key =enrine_key
draw_client_secret = "IAA63QPbaU06h6jA5tVfmpfClmmctXei"
# 改成自己的key
os.environ["EB_AGENT_ACCESS_TOKEN"] = enrine_key

In [31]:
# content='为什么说太阳也有生长老死世上万物皆有生死?辉煌如太阳者亦不能例外。它和众多的恒星一样，也有诞生和死亡。太阳起源于一团以氢分子为主的硕大气体云团。约50亿年前，一团原本十分稀薄的星云在某种外因的影响下开始收缩。由于引力作用，收缩一旦开始就会不断加速。在收缩过程中，动能转变成热能，使得气体温度逐渐升高，随着这团气体的密度越来越大，中心温度越来越高，一旦其核心达到了足够高的温度（约700万开），就会触发热核聚变反应。核心的热核反应“点燃”后，光辐射持续产生的“辐射压”能够有效地抵抗引力的作用，从而使得这团气体不再继续收缩，于是一个稳定的太阳就诞生了。'
# API_key='c490a340c978914d689040a021030e8d932d9d08'
# Q=Question_generae(content,API_key)
# result=Q.generate()
# print(result)

In [32]:
def get_access_token(client_id, client_secret):
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": client_id, "client_secret": client_secret}
    return str(requests.post(url, params=params).json().get("access_token"))
# 生成语音合成任务
def create_task(text:str, client_id:str, client_secret:str):
    url = "https://aip.baidubce.com/rpc/2.0/tts/v1/create?access_token=" + get_access_token(client_id, client_secret)

    payload = json.dumps({
        "text": text,
        "format": "mp3-16k",
        "voice": 110,
        "lang": "zh",
        "speed": 5,
        "pitch": 5,
        "volume": 5,
        "enable_subtitle": 0
    })
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.json())
    return response.json()
# 查询语音任务状态
def get_voise_url(task_id:str, client_id, client_secret):
    url = "https://aip.baidubce.com/rpc/2.0/tts/v1/query?access_token=" + get_access_token(client_id, client_secret)
    payload = json.dumps({
        "task_ids": [task_id]
    })
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    task = response.json()['tasks_info'][0]
    task_status = task['task_status']
    # 如果成功，返回语音地址；如果失败，抛出异常；如果进行中，返回空
    if task_status == 'Success':
        return task['task_result']['speech_url']
    if task_status == 'Failure':
        raise Exception("合成语音失败")
    else:
        return None
# 用于递归语音任务的外部方法
def create_voise(text:str, client_id:str, client_secret:str):
    # 创建语音合成项目
    task_resp = create_task(text, client_id, client_secret)
    task_id = task_resp['task_id']
    # 获取语音文件,循环检测到地址
    voice_url = None
    while voice_url == None:
        voice_url = get_voise_url(task_id, speak_client_id, speak_client_secret)
    return voice_url

# 绘制图片
def draw(answer, api_key, secret_key):
    erniebot.api_type = 'yinian'
    erniebot.ak = api_key
    erniebot.sk = secret_key
    # 注意prompt总字数不得超过100字
    prompt = """基于答案绘制一幅适合小朋友的绘本漫画。
【答案】{}"""
    response = erniebot.Image.create(
        model="ernie-vilg-v2",
        prompt=prompt.format(answer),
        width=512,
        height=512
    )
    return response.get_result()[0]

In [33]:
# import requests
# import json

# API_KEY = ""
# SECRET_KEY = ""

# def main():
        
#     url = "https://aip.baidubce.com/rpc/2.0/tts/v1/create?access_token=" + get_access_token()
    
#     payload = json.dumps({
#         "format": "mp3-16k",
#         "voice": 0,
#         "lang": "zh",
#         "speed": 5,
#         "pitch": 5,
#         "volume": 5,
#         "enable_subtitle": 0
#     })
#     headers = {
#         'Content-Type': 'application/json',
#         'Accept': 'application/json'
#     }
    
#     response = requests.request("POST", url, headers=headers, data=payload)
    
#     print(response.text)
    

# def get_access_token():
#     """
#     使用 AK，SK 生成鉴权签名（Access Token）
#     :return: access_token，或是None(如果错误)
#     """
#     url = "https://aip.baidubce.com/oauth/2.0/token"
#     params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
#     return str(requests.post(url, params=params).json().get("access_token"))

# def main():
        
#     url = "https://aip.baidubce.com/rpc/2.0/tts/v1/query?access_token=" + get_access_token()
    
#     payload = json.dumps("")
#     headers = {
#         'Content-Type': 'application/json',
#         'Accept': 'application/json'
#     }
    
#     response = requests.request("POST", url, headers=headers, data=payload)
    
#     print(response.text)
    

# def get_access_token():
#     """
#     使用 AK，SK 生成鉴权签名（Access Token）
#     :return: access_token，或是None(如果错误)
#     """
#     url = "https://aip.baidubce.com/oauth/2.0/token"
#     params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
#     return str(requests.post(url, params=params).json().get("access_token"))

In [34]:
def generate_click(story):
    # 生成题目
    global Answer
    Q=Question_generae(story,enrine_key)
    result=Q.generate()
    print(result)
    question=[]
    choices=[[] for _ in range(10)]
    Answer=''
    print('successfully start_click')
    i=0
    for QCA in result:
        question.append(QCA[0])
        choices[i].extend(QCA[1:5])
        i+=1
        Answer+=QCA[5][-1]
    print('successfully generate')
    print(question[0])
    print(choices[0])
    Q1=gr.Textbox(label="Question 1",value=question[0])
    A1=gr.Radio(choices[0], label="Choice")
    Q2=gr.Textbox(label="Question 2",value=question[1])
    A2=gr.Radio(choices[1], label="Choice")
    Q3=gr.Textbox(label="Question 3",value=question[2])
    A3=gr.Radio(choices[2], label="Choice")
    Q4=gr.Textbox(label="Question 4",value=question[3])
    A4=gr.Radio(choices[3], label="Choice")
    Q5=gr.Textbox(label="Question 5",value=question[4])
    A5=gr.Radio(choices[4], label="Choice")
    Q6=gr.Textbox(label="Question 6",value=question[5])
    A6=gr.Radio(choices[5], label="Choice")
    Q7=gr.Textbox(label="Question 7",value=question[6])
    A7=gr.Radio(choices[6], label="Choice")
    Q8=gr.Textbox(label="Question 8",value=question[7])
    A8=gr.Radio(choices[7], label="Choice")
    Q9=gr.Textbox(label="Question 9",value=question[8])
    A9=gr.Radio(choices[8], label="Choice")
    Q10=gr.Textbox(label="Question 10",value=question[9])
    A10=gr.Radio(choices[9], label="Choice")   
    print('successfully show')
    return Q1,A1,Q2,A2,Q3,A3,Q4,A4,Q5,A5,Q6,A6,Q7,A7,Q8,A8,Q9,A9,Q10,A10,Answer



def generate_story_click(content):
    print('start generate_story')
    erniebot.api_type = "aistudio"
    erniebot.access_token = enrine_key
    question_id = str(uuid.uuid4()).replace('-','')
    messages=[]
    prompt = f"你是一个富有创意和童心的绘本故事撰写者，可以根据所提供的内容，整理并创作出一篇兼具趣味和叙事性的小故事,下面是提供的内容：{content}"
    messages.append({'role': 'user', 'content': prompt})
    response = erniebot.ChatCompletion.create(
    model='ernie-3.5',
    messages=messages,
    top_p=0.2,
    temperature=0.5,)
    result=response['result']
    print(result)
    story=gr.Textbox(interactive=False,label="Story",value=result)
    print('successfully generate_story')
    # voice_path=Path('./voice/c252908406a347ff804578f0389b2642.mp3')
    # img_path='/img/{}.png'.format(question_id)
    # 生成voice
    result='星空'
    voice_url = create_voise(result, speak_client_id, speak_client_secret)

    voice_path = './voice/{}.mp3'.format(question_id)
    print(f"voic_url={voice_url}")
    wget.download(voice_url, voice_path)
    # voice_path.absolute()
    # # 生成图片
    # img_url = draw(result, draw_api_key, draw_client_secret)
    # img_path = './img/{}.png'.format(question_id)
    # wget.download(img_url, img_path)

    # r = requests.get(img_url)
    # with open(img_path, 'wb') as f:
    #     f.write(r.content)
    audio = gr.Audio(label="Audio",type='filepath',value=voice_path)
    # return story,audio,Image.open(img_path)
    return story,audio



# def checkAnswer(A1,A2,A3,A4,A5,A6,A7,A8,A9,A10):
#     global Answer
#     # 检查答案
#     acc=0
#     if A1[0]==Answer[0]:
#         acc+=1
#     if A2[0]==Answer[1]:
#         acc+=1
#     if A3[0]==Answer[2]:
#         acc+=1
#     if A4[0]==Answer[3]:
#         acc+=1
#     if A5[0]==Answer[4]:
#         acc+=1
#     if A6[0]==Answer[5]:
#         acc+=1
#     if A7[0]==Answer[6]:
#         acc+=1
#     if A8[0]==Answer[7]:
#         acc+=1
#     if A9[0]==Answer[8]:
#         acc+=1
#     if A10[0]==Answer[9]:
#         acc+=1
#     print(A1[0])
#     print(Answer)
#     acc_history.append(acc/10)
#     print(acc_history)

In [35]:
with gr.Blocks() as demo:
    with gr.Tab("Generate Questions"):
        # API_key =gr.Textbox(lines=1, label="API Key", placeholder="Enter your API key")
        with gr.Row():
            content=gr.Textbox(lines=15,label="Content", placeholder="Enter your content",value='为什么说太阳也有生长老死世上万物皆有生死?辉煌如太阳者亦不能例外。它和众多的恒星一样，也有诞生和死亡。太阳起源于一团以氢分子为主的硕大气体云团。约50亿年前，一团原本十分稀薄的星云在某种外因的影响下开始收缩。由于引力作用，收缩一旦开始就会不断加速。在收缩过程中，动能转变成热能，使得气体温度逐渐升高，随着这团气体的密度越来越大，中心温度越来越高，一旦其核心达到了足够高的温度（约700万开），就会触发热核聚变反应。核心的热核反应“点燃”后，光辐射持续产生的“辐射压”能够有效地抵抗引力的作用，从而使得这团气体不再继续收缩，于是一个稳定的太阳就诞生了。')
            story=gr.Textbox(interactive=False,label="Story")
        generate_story=gr.Button("Generate Story")
        generate_QA=gr.Button("Generate Questions")
        with gr.Column():
            audio = gr.Audio(label="语音播放", type='filepath')
            # img = gr.Image(label='绘本', type="pil")
    acc_history=[]
    with gr.Tab("QA"):
        Q1=gr.Textbox(label="Question 1",value='')
        A1=gr.Radio([], label="Choice")
        Q2=gr.Textbox(label="Question 2",value='')
        A2=gr.Radio([], label="Choice")
        Q3=gr.Textbox(label="Question 3",value='')
        A3=gr.Radio([], label="Choice")
        Q4=gr.Textbox(label="Question 4",value='')
        A4=gr.Radio([], label="Choice")
        Q5=gr.Textbox(label="Question 5",value='')
        A5=gr.Radio([], label="Choice")
        Q6=gr.Textbox(label="Question 6",value='')
        A6=gr.Radio([], label="Choice")
        Q7=gr.Textbox(label="Question 7",value='')
        A7=gr.Radio([], label="Choice")
        Q8=gr.Textbox(label="Question 8",value='')
        A8=gr.Radio([], label="Choice")
        Q9=gr.Textbox(label="Question 9",value='')
        A9=gr.Radio([], label="Choice")
        Q10=gr.Textbox(label="Question 10",value='')
        A10=gr.Radio([], label="Choice")
        #generate_story.click(fn=generate_story_click, inputs=[content],outputs=[story, audio, img])   
        generate_story.click(fn=generate_story_click, inputs=[content],outputs=[story, audio])  
        generate_QA.click(fn=generate_click, inputs=[story],outputs=[Q1,A1,Q2,A2,Q3,A3,Q4,A4,Q5,A5,Q6,A6,Q7,A7,Q8,A8,Q9,A9,Q10,A10])
        # check=gr.Button("submit and check")
        # check.click(fn=checkAnswer,inputs=[A1,A2,A3,A4,A5,A6,A7,A8,A9,A10])     
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


start generate_story
**太阳的诞生与成长**

在很久很久以前，宇宙里有一团大大的、蓬松的气体云团，它就像是一个巨大的棉花糖，里面充满了氢分子。这团云团在宇宙中静静地漂浮着，等待着它的命运。

有一天，不知道从哪里来的一股力量，让这团云团开始慢慢收缩。它变得越来越小，越来越紧实。就像我们吹气球一样，越吹越紧，里面的空气就会越来越热。云团也是这样，随着它的收缩，里面的气体变得越来越热，温度越来越高。

当云团中心变得非常非常热，热到了一定的程度，就像火柴被点燃一样，突然之间，云团的核心发生了巨大的变化。这就是热核聚变反应，就像是把小小的石头变成了大大的火花。

这个火花非常明亮，非常温暖，它照亮了周围的一切，就像一个巨大的明灯。这就是我们的太阳，它诞生了！

太阳诞生后，它开始稳定地发光发热，给宇宙带来了光明和温暖。它就像是一个大大的火炉，给周围的行星提供了生存的条件。

但是，就像我们人类一样，太阳也会有老去的一天。虽然它现在看起来很年轻，很辉煌，但总有一天，它的燃料会用完，它的光芒会渐渐暗淡下来。

那时候，太阳会变成一个红巨星，变得更大更红。虽然它不再像现在这么明亮，但它仍然是宇宙中的一颗重要的星星。

所以，小朋友们，你们知道了吗？太阳也有它的生长和衰老，就像我们一样。但是，无论它怎么变化，它都是我们的好朋友，给我们带来了光明和温暖。让我们一起珍惜它，爱护它吧！
successfully generate_story
{'log_id': 17171644476554031, 'task_status': 'Created', 'task_id': '6659d9a05d73020001ca1aa2'}
voic_url=http://aipe-speech.bj.bcebos.com/text_to_speech/2024-05-31/6659d9a05d73020001ca1aa2/speech/0.mp3?authorization=bce-auth-v1%2FALTAKjI91nE52nvtDNRgFlUCVz%2F2024-05-31T14%3A07%3A32Z%2F259200%2F%2F2895c82e61c70a2e47c4baf8113b0f1fc65c52e2c6f6df86d389a9e063301b76
Keybo